In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col,avg,row_number
from snowflake.snowpark import Window

def initiateSession(): 
    
    connection_parameters = {
            "account": "ls88978.eu-west-1",
            "user": "snowpark_user",
            "password": "UdemySnowflake12!@",
            "role": "Accountadmin", 
            "warehouse": "compute_wh"
    }
    session = Session.builder.configs(connection_parameters).create()
    return session

session = initiateSession()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [ ]:
df_orders = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS")

In [ ]:
type(df_orders)

In [ ]:
df_orders.show(5)

In [ ]:
df_orders.groupBy("O_ORDERSTATUS").avg("O_TOTALPRICE").show()

In [ ]:
df_orders.groupBy("O_ORDERSTATUS").count().show()

In [ ]:
df_tmp = session.create_dataframe([("iphone12", "2000","phone","electronic gadgets"), \
                                   ("iphone13", "2400","phone","electronic gadgets"), \
                                   ("sansung galaxy", "2600","phone","electronic gadgets"), \
                                   ("microsoft windows 11", "3600","phone","electronic gadgets"), \
                                   ("samsung galaxy watch", "2600","watch","electronic gadgets"), \
                                   ("samsung galaxy watch2", "2900","watch","electronic gadgets"), \
                                   ("apple watch series 8", "3200","watch","electronic gadgets"), \
                                   ("apple watch se2", "1800","watch","electronic gadgets"), \
                                    ("Nike White Jersey", "300","clothing","non-electronic"), \
                                   ("Air jordon Shoes", "2300","clothing","non-electronic"), \
                                   ("Reebok Sneakers", "3000","clothing","non-electronic"), \
                                   ("yeezy red sneakers", "5300","clothing","non-electronic") \
                                  ], \
                                  schema=["product_name", "value","category","product_type"])

df_tmp.show()

In [ ]:
window = Window.order_by("value") # Without partition by 

window2 = Window.partition_by("product_type").order_by("value") # Using partition by clause

window3 = Window.partition_by(['category','product_type']).order_by("value") # Using multiple columns in partition-by clause


In [ ]:
df_tmp.withColumn("row_number",row_number().over(window)).show()

In [ ]:
df_tmp.withColumn("row_number",row_number().over(window2)).show()

In [ ]:
df_tmp.withColumn("row_number",row_number().over(window3)).show()

In [ ]:
df_orders = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS") \
            .select(['O_ORDERKEY','O_CUSTKEY','O_TOTALPRICE']) \
            .filter("O_ORDERKEY=31593217")

df_orders.show()

In [4]:
sql = """
        select 
            S_SUPPKEY as SUPP_KEY,
            S_NAME as SUPPLIER_NAME,
            S_NATIONKEY as NATION_KEY,
            N.N_NAME as NATION_NAME
        from 
            SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.SUPPLIER S 
        LEFT JOIN 
            SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.NATION N
        ON 
            S.S_NATIONKEY = N.N_NATIONKEY
        """

df_supplier = session.sql(sql)
df_supplier.show()

-------------------------------------------------------------------
|"SUPP_KEY"  |"SUPPLIER_NAME"     |"NATION_KEY"  |"NATION_NAME"   |
-------------------------------------------------------------------
|12001       |Supplier#000012001  |1             |ARGENTINA       |
|12002       |Supplier#000012002  |23            |UNITED KINGDOM  |
|12003       |Supplier#000012003  |13            |JORDAN          |
|12004       |Supplier#000012004  |22            |RUSSIA          |
|12005       |Supplier#000012005  |9             |INDONESIA       |
|12006       |Supplier#000012006  |9             |INDONESIA       |
|12007       |Supplier#000012007  |1             |ARGENTINA       |
|12008       |Supplier#000012008  |21            |VIETNAM         |
|12009       |Supplier#000012009  |3             |CANADA          |
|12010       |Supplier#000012010  |9             |INDONESIA       |
-------------------------------------------------------------------



In [ ]:
type(df_supplier)

In [6]:
import pandas as pd 
df_supplier = df_supplier.to_pandas()
df_supplier.columns = [x.upper() for x in df_supplier.columns]

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [ ]:
type(df_supplier)

In [9]:
query = """create or replace table SNOWPARK_UDEMY.test_schema.SUPPLIER_DETAILS (supp_key integer,supplier_name varchar(100),
            nation_key integer ,nation_name varchar(100) )
        """
session.sql(query).collect()


[Row(status='Table SUPPLIER_DETAILS successfully created.')]

In [21]:
session.sql("use schema SNOWPARK_UDEMY.test_schema").collect()
res = session.write_pandas(df_supplier, "SUPPLIER_DETAILS")

In [25]:
session.close()